In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

# Load cleaned dataset
file_path = "Cleaned_Merged_data_alarm.csv"
df = pd.read_csv(file_path)

# Pisahkan fitur dan target
X = df.drop(columns=["Severity"])  # Fitur
y = df["Severity"]  # Target

# Normalisasi fitur numerik
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Mengatasi Ketidakseimbangan Kelas dengan SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Tampilkan distribusi kelas setelah SMOTE
severity_distribution_resampled = np.bincount(y_train_resampled)
severity_distribution_original = np.bincount(y_train)

print("Distribusi Severity sebelum SMOTE:", severity_distribution_original)
print("Distribusi Severity setelah SMOTE:", severity_distribution_resampled)


Distribusi Severity sebelum SMOTE: [38232  4580  9206 26227]
Distribusi Severity setelah SMOTE: [38232 38232 38232 38232]


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load cleaned dataset
file_path = "Cleaned_Merged_data_alarm.csv"
df = pd.read_csv(file_path)

# Pisahkan fitur dan target
X = df.drop(columns=["Severity"])
y = df["Severity"]

# Normalisasi fitur numerik
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Mengatasi Ketidakseimbangan Kelas dengan SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Simpan dataset yang telah diproses
np.save("X_train.npy", X_train_resampled)
np.save("y_train.npy", y_train_resampled)
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)

print("Dataset train-test telah berhasil disimpan ulang.")


Dataset train-test telah berhasil disimpan ulang.


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Load processed train-test datasets
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

# Reshape for LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Build LSTM model
model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),
    BatchNormalization(),
    LSTM(32, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(4, activation='softmax')  # 4 classes for severity levels
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model with Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, callbacks=[early_stopping])

# Evaluate model
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", report)

# Save model
model.save("lstm_alarm_model.h5")
print("LSTM model saved as lstm_alarm_model.h5")


Epoch 1/50


2025-02-26 12:56:23.704684: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/ubuntu/ryh_training/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4779/4779 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.8011 - loss: 0.5084 - val_accuracy: 0.8890 - val_loss: 0.2974
Epoch 2/50
4779/4779 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8862 - loss: 0.3084 - val_accuracy: 0.9057 - val_loss: 0.2659
Epoch 3/50
4779/4779 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8938 - loss: 0.2888 - val_accuracy: 0.9074 - val_loss: 0.2676
Epoch 4/50
4779/4779 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8984 - loss: 0.2741 - val_accuracy: 0.8924 - val_loss: 0.2868
Epoch 5/50
4779/4779 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9025 - loss: 0.2643 - val_accuracy: 0.9060 - val_loss: 0.2678
Epoch 6/50
4779/4779 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9063 - loss: 0.2543 - val_accuracy: 0.9090 - val_loss: 0.2482
Epoch 7/50
4779/4779 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9069 - loss: 0.2542 - val_accuracy: 0.9160 - val_loss: 0.2502
Epoch 8/50
4779/4779 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9096 - loss: 0.2472 - val_accur

Model Accuracy: 92.43%
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.88      0.93      9558
           1       0.80      0.89      0.84      1145
           2       0.72      0.97      0.83      2302
           3       0.97      0.98      0.98      6557

    accuracy                           0.92     19562
   macro avg       0.87      0.93      0.89     19562
weighted avg       0.94      0.92      0.93     19562

LSTM model saved as lstm_alarm_model.h5


In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from datetime import datetime

# Load trained model
model = tf.keras.models.load_model("lstm_alarm_model.h5")

# Cek jumlah fitur yang digunakan saat training
df = pd.read_csv("Cleaned_Merged_data_alarm.csv")
num_features = df.drop(columns=["Severity"]).shape[1]  # Ambil jumlah fitur yang benar

# Generate dummy real-time data dengan jumlah fitur yang benar
num_samples = 10  # Jumlah sampel baru
dummy_data = np.random.rand(num_samples, num_features)  # Data acak antara 0-1

# Normalisasi dengan StandardScaler (gunakan scaler yang sama dari training)
scaler = StandardScaler()
dummy_data_scaled = scaler.fit_transform(dummy_data)  

# Reshape untuk LSTM
X_real_time = dummy_data_scaled.reshape((num_samples, 1, num_features))

# Predict
y_real_time_pred = np.argmax(model.predict(X_real_time), axis=1)

# Generate real-time timestamp
timestamps = [datetime.now().strftime("%Y-%m-%d %H:%M:%S") for _ in range(num_samples)]

# Generate dummy Alarm ID (random values between 1-1000)
alarm_ids = np.random.randint(1, 1000, size=num_samples)

# Convert to DataFrame
df_real_time = pd.DataFrame({
    "Timestamp": timestamps,
    "Predicted Severity": y_real_time_pred,
    "Alarm ID": alarm_ids
})


# Print hasil prediksi
print("Real-time Predictions (Filtered):")
print(df_real_time.head())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
Real-time Predictions (Filtered):
             Timestamp  Predicted Severity  Alarm ID
0  2025-02-26 13:09:14                   1       600
1  2025-02-26 13:09:14                   3       850
2  2025-02-26 13:09:14                   1       126
3  2025-02-26 13:09:14                   1       721
4  2025-02-26 13:09:14                   1       559


In [15]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta

# Load trained model
model = tf.keras.models.load_model("lstm_alarm_model.h5")

# Cek jumlah fitur yang digunakan saat training
df = pd.read_csv("Cleaned_Merged_data_alarm.csv")
num_features = df.drop(columns=["Severity"]).shape[1]  # Ambil jumlah fitur yang benar

# Load real-time data terbaru
latest_data = np.random.rand(1, num_features)  # Dummy real-time data

# Normalisasi dengan StandardScaler (gunakan scaler yang sama dari training)
scaler = StandardScaler()
latest_data_scaled = scaler.fit_transform(latest_data)  

# Reshape untuk LSTM
X_future = latest_data_scaled.reshape((1, 1, num_features))

# Predict severity di masa depan
y_future_pred = np.argmax(model.predict(X_future), axis=1)[0]

# Prediksi waktu di masa depan (misalnya 1 jam ke depan)
future_timestamp = (datetime.now() + timedelta(hours=1)).strftime("%Y-%m-%d %H:%M:%S")

# Generate dummy Alarm ID untuk prediksi
future_alarm_id = np.random.randint(1, 1000)

# Buat DataFrame untuk hasil prediksi
df_future = pd.DataFrame({
    "Timestamp": [future_timestamp],
    "Predicted Severity": [y_future_pred],
    "Alarm ID": [future_alarm_id]
})


# Print hasil prediksi
print("Predicted Future Alarm:")
print(df_future)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
Predicted Future Alarm:
             Timestamp  Predicted Severity  Alarm ID
0  2025-02-26 14:11:22                   3        22
